In [1]:
import os
import numpy as np
import music21 as m21
import pandas as pd
import json
import matplotlib.pyplot as plt
import time

np.random.seed(777)

## Functions

In [2]:
DIV_CONST = 4

In [3]:
def getSongKey(song):
    key = song.analyze("key")
    return key

In [4]:
def getSongKeyFromMelody_W_Times(melody_w_times_in_k):
    sc_test = m21.stream.Score()
    p0_test = m21.stream.Part()
    p0_test.id = 'part0'
    for pitch_i in melody_w_times_in_k:
        n_i = m21.note.Note(pitch_i[4])
        p0_test.append(n_i)
    sc_test.insert(0, p0_test)
    return getSongKey(sc_test)

In [5]:
# Function to retrieve a list of midi pitch events and its timestamp
def getMelodyDeltaTimes(eventsintrack):
    
    # Initialize array
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Traverse the events
    for ev in eventsintrack:
        
        # If a note starts
        if (ev.isNoteOn()):
        
            
            # Get the pitch name and save it with the cumulative sum, midi pitch and name
            pitch_in_time = m21.pitch.Pitch(ev.pitch)
            DeltaTimes.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish, pitch_in_time))
            
            # Restart the delta time
            prev_deltatime = 0
        
        # Else if there is a delta time
        elif(str(ev.type) == "DeltaTime"):
          
            # We sum the time
            cum_sum += ev.time
            
            # We sum it to the current delta time
            prev_deltatime += ev.time
    
    # Return the array
    return DeltaTimes

In [6]:
def get_SCLM_v100(melody_w_times_A, melody_w_times_B):
    
    # We use a Dynamic Programming approach
    max_len = max(len(melody_w_times_A), len(melody_w_times_B)) + 1
    
    # memoization array
    memo = np.full(shape=(max_len,max_len), fill_value=-1)
    
    # Get the limits for each melody
    lim_A = len(melody_w_times_A)
    lim_B = len(melody_w_times_B)
    
    # Actual DP implementation
    for i in range(lim_A, -1, -1):
        for j in range(lim_B, -1, -1):
            
            # If we are at the limits the solution is 0
            if i == lim_A or  j == lim_B:
                memo[i][j] = 0
                continue
            
            # If there is a match a possible solution is the previous plus one
            curr_value = 0
            
            tot_delta_time = (float(melody_w_times_A[i][1]) + float(melody_w_times_B[j][1])) / float(DIV_CONST)
            tot_diff_time = np.abs(float(melody_w_times_A[i][1]) - float(melody_w_times_B[j][1]))
            
            
            if (melody_w_times_A[i][3] == melody_w_times_B[j][3]) and (tot_diff_time <= tot_delta_time):
                curr_value = memo[i + 1][j + 1] + 1
                
            # The actual solution is the maximum between the one if there is a match, or skip on the melody A or melody B
            curr_value = max(curr_value, max(memo[i + 1][j], memo[i][j + 1]))
            
            # Save the solution
            memo[i][j] = curr_value
    
    # With the memoization table we can retrieve the actual melody
    i = 0
    j = 0
    SCLM = []
    while i != lim_A and j != lim_B:
    
        if ((memo[i + 1][j + 1] + 1) == memo[i][j]):
            SCLM.append((i, j))
            i += 1
            j += 1
        elif (memo[i + 1][j] == memo[i][j]):
            i += 1
        elif (memo[i][j + 1] == memo[i][j]):
            j += 1
    
    return SCLM

In [7]:
def get_max_timestamp_dif(melody_w_times_A, melody_w_times_B):
    return max(
        melody_w_times_A[len(melody_w_times_A) - 1][0] - melody_w_times_A[0][0],
        melody_w_times_B[len(melody_w_times_B) - 1][0] - melody_w_times_B[0][0]
    )

In [8]:
def getDifSCLM(melody_w_times_A, melody_w_times_B, sclm):
    
    # If there is no sclm or it is just one return max possible value
    if (len(sclm) <= 1):
        return get_max_timestamp_dif(melody_w_times_A, melody_w_times_B)
    
    
    # Initialize the arrays
    T_A = np.zeros(shape=(len(sclm) - 1))
    T_B = np.zeros(shape=(len(sclm) - 1))
    T_C = np.zeros(shape=(len(sclm) - 1))
    Dif_ = np.zeros(shape=(len(sclm) - 1))
    
    for i in range(1, len(sclm)):
        T_A[i - 1] = melody_w_times_A[sclm[i][0]][0] - melody_w_times_A[sclm[i-1][0]][0]
        T_B[i - 1] = melody_w_times_B[sclm[i][1]][0] - melody_w_times_B[sclm[i-1][1]][0]
        T_C[i - 1] = np.abs(T_A[i - 1] - T_B[i - 1])
    
    T_C_mean = np.mean(T_C)
    
    for i in range(0, len(T_B)):
        T_B[i] += T_C_mean
        Dif_[i] = T_A[i] - T_B[i]
    
    return T_C_mean
    

In [9]:
def get_MTRC_v100_from_melody_w_times(melody_w_times_A, melody_w_times_B):
    
    # Assert at least one element for each melody
    if (len(melody_w_times_A) == 0 or len(melody_w_times_B) == 0):
        print("EMPTY")
        return 1
    
    # Initialize result variable
    result_value = 0
    
    # Get Keys
    key_A = getSongKeyFromMelody_W_Times(melody_w_times_A)
    key_B = getSongKeyFromMelody_W_Times(melody_w_times_B)
    
    # D1: Scale  
    scale_dif1 = 0
    if (key_A.name != key_B.name):
        scale_dif1 = W1
    result_value += scale_dif1
    
    # D2: Mode  
    mode_dif2 = 0
    if (key_A.mode != key_B.mode):
        mode_dif2 = W2
    result_value += mode_dif2
    
    # Get SCLM v100
    sclm = get_SCLM_v100(melody_w_times_A, melody_w_times_B)
    
    # Get max len
    max_len = max(len(melody_w_times_A), len(melody_w_times_B))
    
    # D3: SCLM Length
    sclmlen_dif3 = ((max_len - len(sclm)) / max_len) * W3
    result_value += sclmlen_dif3
    
    # Get the Diff on temporal spacing in the SCLM
    dif_sclm = getDifSCLM(melody_w_times_A, melody_w_times_B, sclm)
    
    # D4: dif in sclm
    max_timestamp_dif = get_max_timestamp_dif(melody_w_times_A, melody_w_times_B)
    sclmdif_dif4 = (dif_sclm / max_timestamp_dif) * W4
    result_value += sclmdif_dif4
    
    return result_value

## Traverse DATA

In [10]:
## NES ##
NES_DATASET_PATH = "/media/sirivasv/JASON/Saul/MCC/DATASETS/DATASUBSET/nesmdb_midi/"

In [11]:
# Traverse midi files
nes_song_filenames = []
for root, directories, files in os.walk(NES_DATASET_PATH):
    for file in files:
        nes_song_filenames.append(file)
print(nes_song_filenames[:3])
print(len(nes_song_filenames))

['134_GanbareGoemon2_17_18DeathbyBoilingHell.mid', '000_10_YardFight_00_01GameStart.mid', '000_10_YardFight_01_02GameOver.mid']
5278


In [12]:
%%time

W1 = 0.0
W2 = 0.0
W3 = 1.0
W4 = 0.0

# Read Files 
MAX_LIM_NES_SONGS = len(nes_song_filenames)
len_nes_song_filenames = len(nes_song_filenames)
nes_songs_with_error = []
nes_similarities_for_sort = []


# Query File
# "322_SuperMarioBros__02_03SwimmingAround.mid"
# "322_SuperMarioBros__10_11SavedthePrincess.mid"
# "339_Tetris_00_01TitleScreen.mid"
song_filename_query = "322_SuperMarioBros__03_04BowsersCastle.mid"
song_stream_query = m21.converter.parseFile(os.path.join(NES_DATASET_PATH, song_filename_query))
midi_tracks_query = m21.midi.translate.streamToMidiFile(song_stream_query)
melody_w_times_query = getMelodyDeltaTimes(midi_tracks_query.tracks[1].events)

# We traverse the reduced table
cnt = 1
for song_filename_test in nes_song_filenames:
    try:
        
        song_stream_test = m21.converter.parseFile(os.path.join(NES_DATASET_PATH, song_filename_test))
        midi_tracks_test = m21.midi.translate.streamToMidiFile(song_stream_test)
        melody_w_times_test = getMelodyDeltaTimes(midi_tracks_test.tracks[1].events)

        similarity_distance = get_MTRC_v100_from_melody_w_times(
            melody_w_times_query,
            melody_w_times_test)
        
        nes_similarities_for_sort.append((song_filename_test, similarity_distance))
        
        with open('./PROP1_query_to_sort_{0}.json'.format(song_filename_query.split(".")[0]), 'w') as outfile:
            json.dump({"data":nes_similarities_for_sort}, outfile)
        
    except:
        print("[ERROR!]")
        nes_songs_with_error.append(song_filename_test)
    finally:
        print("{0}/{1} - {2} - {3}".format(cnt, len_nes_song_filenames, song_filename_test, similarity_distance))
        cnt += 1
        if (cnt == MAX_LIM_NES_SONGS):
            break
    

1/5278 - 134_GanbareGoemon2_17_18DeathbyBoilingHell.mid - 0.8
2/5278 - 000_10_YardFight_00_01GameStart.mid - 0.9615384615384616
3/5278 - 000_10_YardFight_01_02GameOver.mid - 0.9615384615384616
4/5278 - 001_1942_00_01Start.mid - 1.0
5/5278 - 001_1942_01_02MainBGM.mid - 0.9711538461538461
6/5278 - 001_1942_02_03Restart.mid - 1.0
7/5278 - 001_1942_03_04MainBGMRestart.mid - 0.9711538461538461
8/5278 - 001_1942_04_05StageClear.mid - 0.9519230769230769
9/5278 - 001_1942_05_06GameOver.mid - 0.9711538461538461
10/5278 - 002_1943_TheBattleofMidway_00_01Title.mid - 0.9038461538461539
11/5278 - 002_1943_TheBattleofMidway_01_02PowerUpYourP38.mid - 0.9423076923076923
12/5278 - 002_1943_TheBattleofMidway_02_03MissionStart.mid - 0.9903846153846154
13/5278 - 002_1943_TheBattleofMidway_03_04AirBattleA.mid - 0.861271676300578
14/5278 - 002_1943_TheBattleofMidway_04_05AntishipBattleA.mid - 0.8591549295774648
15/5278 - 002_1943_TheBattleofMidway_05_06MissionCompletedI.mid - 0.9326923076923077
16/5278 - 00

114/5278 - 085_DragonBusterII_YaminoFuuin_13_14KingDragonClear.mid - 0.9038461538461539
115/5278 - 085_DragonBusterII_YaminoFuuin_14_15Ending.mid - 0.9615384615384616
116/5278 - 086_DragonFighter_00_01Title.mid - 0.868766404199475
117/5278 - 086_DragonFighter_01_02TheFrozenBattleStage1.mid - 0.8876080691642652
118/5278 - 086_DragonFighter_02_03Boss.mid - 0.8815165876777251
119/5278 - 086_DragonFighter_03_04StageClear.mid - 0.8103448275862069
120/5278 - 086_DragonFighter_04_05DontDisturbStage2.mid - 0.8856304985337243
121/5278 - 086_DragonFighter_05_06IntotheDepthStage3.mid - 0.9005145797598628
122/5278 - 086_DragonFighter_06_07CanyouChainUpStage4.mid - 0.8846153846153846
123/5278 - 086_DragonFighter_07_08InitiveofaGhostStage5.mid - 0.9051383399209486
124/5278 - 086_DragonFighter_08_09TheDragonFighterStage6.mid - 0.8996763754045307
125/5278 - 086_DragonFighter_09_10FinalBoss.mid - 0.8824884792626728
126/5278 - 086_DragonFighter_10_11Ending.mid - 0.9478584729981379
127/5278 - 116_Felixth

218/5278 - 178_Ironsword_Wizards_amp_WarriorsII_06_07Sky.mid - 0.9513513513513514
219/5278 - 178_Ironsword_Wizards_amp_WarriorsII_07_08WindElemental.mid - 0.89937106918239
220/5278 - 178_Ironsword_Wizards_amp_WarriorsII_08_09AcquireSwordPart.mid - 0.8557692307692307
221/5278 - 194_KnightMove_04_05Umasakaba.mid - 0.918918918918919
222/5278 - 194_KnightMove_05_06HighScoreBoard.mid - 0.9612403100775194
223/5278 - 194_KnightMove_06_07ChampionScore.mid - 0.9065420560747663
224/5278 - 194_KnightMove_07_08Congratulations.mid - 0.9807692307692307
225/5278 - 194_KnightMove_08_092PlayersWin.mid - 0.8434782608695652
226/5278 - 195_KonamiWaiWaiWorld_00_01GarageBGM.mid - 0.7692307692307693
227/5278 - 195_KonamiWaiWaiWorld_01_02StageSelectBGM.mid - 0.8846153846153846
228/5278 - 195_KonamiWaiWaiWorld_02_03KonamiManBGM.mid - 0.8709677419354839
229/5278 - 195_KonamiWaiWaiWorld_03_04KonamiLadyBGM.mid - 0.9154929577464789
230/5278 - 195_KonamiWaiWaiWorld_04_05GoemonBGM.mid - 0.8449612403100775
231/5278 -

331/5278 - 291_ShadowoftheNinja_03_04Boss.mid - 0.8235294117647058
332/5278 - 291_ShadowoftheNinja_04_05StageClear.mid - 0.7596153846153846
333/5278 - 291_ShadowoftheNinja_05_06Stage2.mid - 0.920863309352518
334/5278 - 291_ShadowoftheNinja_06_07Stage3.mid - 0.8974789915966387
335/5278 - 291_ShadowoftheNinja_07_08Stage4.mid - 0.8888888888888888
336/5278 - 338_TerraCresta_03_04LastStart.mid - 0.9615384615384616
337/5278 - 338_TerraCresta_04_05Tubo.mid - 0.8709677419354839
338/5278 - 338_TerraCresta_05_06Dycoon.mid - 0.8199052132701422
339/5278 - 338_TerraCresta_06_07Mandler.mid - 0.9140461215932913
340/5278 - 338_TerraCresta_07_08GameOver.mid - 0.9519230769230769
341/5278 - 338_TerraCresta_08_09MarchingRaster.mid - 0.88
342/5278 - 338_TerraCresta_09_10EditMode.mid - 0.87248322147651
343/5278 - 338_TerraCresta_10_11Unknown.mid - 0.7962962962962963
344/5278 - 339_Tetris_00_01TitleScreen.mid - 0.9134615384615384
345/5278 - 339_Tetris_01_02Loginska.mid - 0.9552238805970149
346/5278 - 339_Tet

446/5278 - 018_ArumananoKiseki_11_12GameOver.mid - 0.9615384615384616
447/5278 - 020_Athena_01_02CloseTheScene.mid - 0.8653846153846154
448/5278 - 020_Athena_02_03OpenTheScene.mid - 0.8846153846153846
449/5278 - 020_Athena_03_04WorldofForestStage1Ending.mid - 0.8802395209580839
450/5278 - 020_Athena_04_05WorldofCavernStage2.mid - 0.8076923076923077
451/5278 - 020_Athena_05_06WorldofSeaStage3.mid - 0.8653846153846154
452/5278 - 020_Athena_06_07WorldofSkyStage4.mid - 0.868421052631579
453/5278 - 020_Athena_07_08WorldofIceStage5.mid - 0.8571428571428571
454/5278 - 020_Athena_08_09WorldofHellStage6.mid - 0.8076923076923077
455/5278 - 020_Athena_09_10WorldofLabyrinthStage7.mid - 0.8653846153846154
456/5278 - 020_Athena_10_11Boss.mid - 0.8173076923076923
457/5278 - 020_Athena_11_12Death.mid - 0.8942307692307693
458/5278 - 020_Athena_12_13GameOver.mid - 0.9038461538461539
459/5278 - 021_AtlantisnoNazo_01_02GameStart.mid - 0.9519230769230769
460/5278 - 021_AtlantisnoNazo_02_03MainBGM1.mid - 0.

...

4781/5278 - 389_WreckingCrew_03_04GameOver.mid - 0.8942307692307693
4782/5278 - 389_WreckingCrew_04_05GoldenHammer.mid - 0.9554285714285714
4783/5278 - 390_Xevious_00_01GameStart.mid - 0.9326923076923077
4784/5278 - 390_Xevious_01_02BGM.mid - 0.9903846153846154
4785/5278 - 391_YieArKungFu_00_01MainBGM.mid - 0.7888888888888889
4786/5278 - 391_YieArKungFu_01_02YouWin.mid - 0.9711538461538461
4787/5278 - 391_YieArKungFu_02_03GameOver.mid - 0.9519230769230769
4788/5278 - 392_Yo_Noid_00_01TitleScreen.mid - 0.875
4789/5278 - 392_Yo_Noid_01_02TitleScreen.mid - 0.9326923076923077
4790/5278 - 392_Yo_Noid_02_03Map.mid - 0.9134615384615384
4791/5278 - 392_Yo_Noid_03_04Map.mid - 0.9326923076923077
4792/5278 - 392_Yo_Noid_04_05PierIceSupermarket.mid - 0.8279569892473119
4793/5278 - 392_Yo_Noid_05_06IslandIceIsland.mid - 0.8767772511848341
4794/5278 - 392_Yo_Noid_07_08CardDuel.mid - 0.7692307692307693
4795/5278 - 392_Yo_Noid_08_09StageClear.mid - 0.9134615384615384
4796/5278 - 392_Yo_Noid_09_10Stage

4889/5278 - 397_YsII_AncientYsVanishedTheFinalChapter_19_20PressureRoad.mid - 1.0
4890/5278 - 397_YsII_AncientYsVanishedTheFinalChapter_20_21DontGoSoSmoothly.mid - 0.8269230769230769
4891/5278 - 397_YsII_AncientYsVanishedTheFinalChapter_21_22Feena.mid - 0.9519230769230769
4892/5278 - 397_YsII_AncientYsVanishedTheFinalChapter_22_23Termination.mid - 0.8847583643122676
4893/5278 - 397_YsII_AncientYsVanishedTheFinalChapter_23_24AStillTime.mid - 0.8653846153846154
4894/5278 - 397_YsII_AncientYsVanishedTheFinalChapter_24_25StayWithMeForever.mid - 0.9426751592356688
4895/5278 - 397_YsII_AncientYsVanishedTheFinalChapter_26_27LuckyTreasureBox.mid - 0.9326923076923077
4896/5278 - 398_Ys_AncientYsVanishedOmen_00_01Feena.mid - 0.9159663865546218
4897/5278 - 398_Ys_AncientYsVanishedOmen_01_02FountainofLove.mid - 0.8942307692307693
4898/5278 - 398_Ys_AncientYsVanishedOmen_02_03HouseShop.mid - 0.8365384615384616
4899/5278 - 398_Ys_AncientYsVanishedOmen_03_04SarasHouse.mid - 0.9615384615384616
4900/52

4991/5278 - 079_DoubleDragonIII_TheSacredStones_18_19RoppetheStonemanBoss52.mid - 0.8896551724137931
4992/5278 - 079_DoubleDragonIII_TheSacredStones_19_20ToChildrenintheWorldEnding.mid - 0.9295774647887324
4993/5278 - 079_DoubleDragonIII_TheSacredStones_20_21GameOver.mid - 0.9615384615384616
4994/5278 - 080_DoubleDribble_00_01Start.mid - 0.9615384615384616
4995/5278 - 080_DoubleDribble_01_02USAAnthem.mid - 0.8141592920353983
4996/5278 - 080_DoubleDribble_02_03Options.mid - 0.9038461538461539
4997/5278 - 080_DoubleDribble_03_04Scored1.mid - 0.9519230769230769
4998/5278 - 080_DoubleDribble_04_05Scored2.mid - 0.9423076923076923
4999/5278 - 080_DoubleDribble_05_06Scored3.mid - 0.9326923076923077
5000/5278 - 080_DoubleDribble_06_07HalfTime.mid - 0.8698224852071006
5001/5278 - 080_DoubleDribble_07_08GameSet.mid - 0.8166666666666667
5002/5278 - 081_Dr_JekyllandMr_Hyde_00_01Title.mid - 0.9807692307692307
5003/5278 - 115_FelixtheCat_02_03Level2.mid - 0.9756205673758865
5004/5278 - 115_FelixtheC

5094/5278 - 220_MarbleMadness_01_02Level2.mid - 0.8695652173913043
5095/5278 - 220_MarbleMadness_02_03Level3.mid - 0.925531914893617
5096/5278 - 257_OverHorizon_11_12StaffRoll.mid - 0.8778625954198473
5097/5278 - 257_OverHorizon_12_13Ending.mid - 0.8758169934640523
5098/5278 - 257_OverHorizon_13_14GameOver.mid - 1.0
5099/5278 - 258_P_O_W__PrisonersofWar_00_01Title.mid - 0.8557692307692307
5100/5278 - 258_P_O_W__PrisonersofWar_01_02Start.mid - 0.7307692307692307
5101/5278 - 258_P_O_W__PrisonersofWar_02_03Stage1.mid - 0.8843283582089553
5102/5278 - 258_P_O_W__PrisonersofWar_03_04Room1.mid - 0.8787878787878788
5103/5278 - 258_P_O_W__PrisonersofWar_04_05Boss1.mid - 0.8269230769230769
5104/5278 - 258_P_O_W__PrisonersofWar_05_06StageClear.mid - 0.9519230769230769
5105/5278 - 258_P_O_W__PrisonersofWar_06_07Stage2.mid - 0.8478260869565217
5106/5278 - 258_P_O_W__PrisonersofWar_07_08Boss2.mid - 0.8653846153846154
5107/5278 - 258_P_O_W__PrisonersofWar_08_09Stage3.mid - 0.9049586776859504
5108/527

5201/5278 - 369_VS_NinjaJaJaMaru_kun_08_09StageClear.mid - 0.9807692307692307
5202/5278 - 369_VS_NinjaJaJaMaru_kun_09_10PerfectStageClear.mid - 0.9519230769230769
5203/5278 - 369_VS_NinjaJaJaMaru_kun_10_11SuccessfulRescueofPrincessSakura.mid - 0.9711538461538461
5204/5278 - 369_VS_NinjaJaJaMaru_kun_11_12Miss.mid - 0.9423076923076923
5205/5278 - 369_VS_NinjaJaJaMaru_kun_12_13GameOver.mid - 0.9519230769230769
5206/5278 - 369_VS_NinjaJaJaMaru_kun_13_14NameEntry.mid - 0.9326923076923077
5207/5278 - 370_ValkyrienoBouken_TokinoKagiDensetsu_00_01BGMAOverground.mid - 0.8461538461538461
5208/5278 - 370_ValkyrienoBouken_TokinoKagiDensetsu_01_02LevelUp.mid - 0.9423076923076923
5209/5278 - 370_ValkyrienoBouken_TokinoKagiDensetsu_02_03Poison.mid - 0.9326923076923077
5210/5278 - 370_ValkyrienoBouken_TokinoKagiDensetsu_03_04Danger.mid - 0.9903846153846154
5211/5278 - 370_ValkyrienoBouken_TokinoKagiDensetsu_04_05MissGameOver.mid - 0.7788461538461539
5212/5278 - 394_Yoshi_sCookie_08_09StageClear.mid - 

In [13]:
nes_similarities_for_sort

[('134_GanbareGoemon2_17_18DeathbyBoilingHell.mid', 0.8),
 ('000_10_YardFight_00_01GameStart.mid', 0.9615384615384616),
 ('000_10_YardFight_01_02GameOver.mid', 0.9615384615384616),
 ('001_1942_00_01Start.mid', 1.0),
 ('001_1942_01_02MainBGM.mid', 0.9711538461538461),
 ('001_1942_02_03Restart.mid', 1.0),
 ('001_1942_03_04MainBGMRestart.mid', 0.9711538461538461),
 ('001_1942_04_05StageClear.mid', 0.9519230769230769),
 ('001_1942_05_06GameOver.mid', 0.9711538461538461),
 ('002_1943_TheBattleofMidway_00_01Title.mid', 0.9038461538461539),
 ('002_1943_TheBattleofMidway_01_02PowerUpYourP38.mid', 0.9423076923076923),
 ('002_1943_TheBattleofMidway_02_03MissionStart.mid', 0.9903846153846154),
 ('002_1943_TheBattleofMidway_03_04AirBattleA.mid', 0.861271676300578),
 ('002_1943_TheBattleofMidway_04_05AntishipBattleA.mid', 0.8591549295774648),
 ('002_1943_TheBattleofMidway_05_06MissionCompletedI.mid', 0.9326923076923077),
 ('002_1943_TheBattleofMidway_06_07FailedintheAttack.mid', 0.9230769230769231)

In [14]:
nes_similarities_for_sort.sort(key=lambda x: x[1])

In [15]:
nes_similarities_for_sort

[('322_SuperMarioBros__03_04BowsersCastle.mid', 0.0),
 ('322_SuperMarioBros__14_15BowsersCastleHurry.mid', 0.28846153846153844),
 ('158_HikarinoSenshiPhoton_WakuseiZoldiasnoTatakai_17_18Area44.mid',
  0.6101694915254238),
 ('083_DraculaII_NoroinoFuuin_00_01MessageofDarknessNamingBGM.mid',
  0.6826923076923077),
 ('401_ZeldaII_TheAdventureofLink_11_12GameOver.mid', 0.6981132075471698),
 ('312_Stinger_09_10Invincibility.mid', 0.7115384615384616),
 ('047_CastlevaniaII_Simon_sQuest_07_08MessageofDarkness.mid',
  0.7123287671232876),
 ('286_Rygar_18_19LigarLastBoss.mid', 0.719626168224299),
 ('217_MajouDensetsuII_DaimashikyouGalious_07_08GreatDemon.mid', 0.72),
 ('334_Target_Renegade_00_01TaitoLogo.mid', 0.7211538461538461),
 ('008_AfterBurnerII_05_06City202.mid', 0.7232142857142857),
 ('042_CaptainTsubasaVol_II_SuperStriker_06_07SaoPauloFC.mid',
  0.7232142857142857),
 ('147_Gradius_04_05BlankMask.mid', 0.7307692307692307),
 ('183_Karnov_06_07AllCleared.mid', 0.7307692307692307),
 ('258_P_

In [16]:
with open('./PROP1_query_sorted_{0}.json'.format(song_filename_query.split(".")[0]), 'w') as outfile:
            json.dump({"data":nes_similarities_for_sort}, outfile)